In [3]:
from datasets import  load_dataset

dataset = load_dataset("ranWang/UN_PDF_TEXT_DATA", split="randomTest")

Found cached dataset parquet (C:/Users/wRan/.cache/huggingface/datasets/ranWang___parquet/ranWang--UN_PDF_TEXT_DATA-8444a7274238e7fd/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


In [4]:
def get_all_table_contents(article_content):
    table_contents_line = []
    
    for line in article_content.split("\n"):
        if "....." in line:
            table_contents_line.append(line)
    
    return list(map(lambda s:s[:s.find("..")].strip(), table_contents_line)) if len(table_contents_line) > 6 else None

In [5]:
def preprocessin_file_content(file_content:str):
    page_file_content_list = file_content.split("\n----\n")
    
    return_list = []
    for page_file_content in page_file_content_list:
        page_file_content_split = page_file_content.split("\n")
        
        body_line_last_index = len(page_file_content_split)
        for index, page_file_content_line in enumerate(page_file_content_split):
            if "__________" in page_file_content_line:
                body_line_last_index = index
                break
        
        return_list.append("\n".join(page_file_content_split[3:body_line_last_index]).strip())
        
        
    return "\n".join(return_list)

In [6]:
preprocessin_file_content(dataset[1]["zh"])

'07-28459 (C)    130407    180407 \n*0728459*  \n给予殖民地国家和人民独立宣言 \n执行情况特别委员会 \n  百慕大 \n  秘书处编写的工作文件 \n目录 \n 段次页次\n一. 概况.......................................................... .... 1-43\n二. 宪政、法律和政治问题 .............................................. 5-16 3\n三. 预算.......................................................... .... 17-19 5\n四. 经济状况........................................................ ..20-45 5\nA. 概况..........................................................20-24 5\nB. 金融服务......................................................25-26 6\nC. 旅游业........................................................27-32 7\nD. 建筑业........................................................33-36 7\nE. 交通和通信....................................................37-45 8\n五. 社会状况........................................................ ..46-83 9\nA. 概况.......................................................... 46-49 9\nB. 劳工.......................................................... 50-57 10\nC. 教育..........

In [7]:
def slicing_articles_by_table_contents(article_content:str, table_contents:list):
    body_text = article_content.replace("\n----\n", "").split(table_contents[-1])[1]
    
    table_contents = list(map(lambda s:s.replace(" ",""), table_contents))
    
    body_text_line_list = [s.strip() for s in body_text.split("\n") if s.strip() != '']
    effective_body_text_line_list = list(filter(lambda x: not "_____" in x and not "......." in x, body_text_line_list))
    
    table_split_body_text_line_list = []
    table_index = 0
    
    per_insert_table_str = ""
    for effective_body_text_line in effective_body_text_line_list:
        if table_contents[table_index] in effective_body_text_line.replace(" ",""):
            table_split_body_text_line_list.append(per_insert_table_str)
            per_insert_table_str = ""
            table_index += 1
            
        per_insert_table_str += effective_body_text_line + "\n"
        
    table_split_body_text_line_list.append(per_insert_table_str)
    
    table_split_body_text_line_list = table_split_body_text_line_list[1:]
    

    return_list = []
    insetrt_str = ""
    for table_split_body_text in table_split_body_text_line_list:
        if len(table_split_body_text.split("\n")) <=2:
            insetrt_str += table_split_body_text
        else:
            return_list.append(insetrt_str + table_split_body_text)
            insetrt_str = ""
    
    return return_list

In [8]:
slicing_articles_by_table_contents(preprocessin_file_content(dataset[5]["zh"]), get_all_table_contents(dataset[5]["zh"]))

[]

In [9]:
import re

def post_processing(table_split_article_content:list, table_contents:list):
    table_contents = list(map(lambda s:s.replace(" ",""), table_contents))
    
    paragraph_list = []
    paragraph = ""
    index = 1
    for article_content in table_split_article_content:
        article_content_split = article_content.split("\n")
        for s in article_content_split:
            if s.replace(" ","") in table_contents:
                paragraph_list.append(paragraph)
                paragraph_list.append(s)
                paragraph = ""
            elif re.search(r"^\d+\.", s) and int(s.split(".")[0]) == index:
                index += 1
                paragraph_list.append(paragraph)
                paragraph = s
            else:
                paragraph += s

    return list(filter(lambda x:not x in "", paragraph_list))



def find_last_sequence(index, table_split_article_content):
    return_number = None
    for article_content_line in table_split_article_content[index+1:]:
        if re.search(r"^\d+\.", article_content_line):
            return_number = article_content_line.split(".")[0]
            break
            
    return return_number

def spilt_article_content(line):
    paragraph_index = int(line[:line.find(".")]) + 1
    return_list = []
    
    while True:
        line_split = line.split(f"{paragraph_index}.")
        if len(line_split) == 2:
            return_list.append(line_split[0])
            line = f"{paragraph_index}.{line_split[1]}"
        if len(line_split) == 1:
            line_split = line.split(f"{paragraph_index}．")
            if len(line_split) == 2:
                return_list.append(line_split[0])
                line = f"{paragraph_index}.{line_split[1]}"
            else:
                return_list.append(line_split[0])
                break
                
        paragraph_index = paragraph_index + 1
        
    return return_list
            

def post_processing1(table_split_article_content:list):
    return_list = []
    for index, article_content_line in enumerate(table_split_article_content):
        match = re.search(r"^\d+\.", article_content_line)
        if match:
            
            last_table_index = find_last_sequence(index+1, table_split_article_content)
            
            if last_table_index == None:
                return_list.append(article_content_line)
            else:
                if last_table_index == str(int(match[0][:-1]) + 1):
                    return_list.append(article_content_line)
                else:
                    for s in spilt_article_content(article_content_line):
                        if s:
                            return_list.append(s)
                
        else:
            return_list.append(article_content_line)
            
    return return_list

In [10]:
def main(text):
    table_contents = get_all_table_contents(text)
    
    text_list = slicing_articles_by_table_contents(preprocessin_file_content(text), table_contents)
    
    return list(filter(lambda x:x!="", post_processing1(post_processing(text_list, table_contents))))

In [11]:
main(dataset[5]["zh"])

[]

In [12]:
len(dataset[5]["zh"].replace("\n----\n", "").split(get_all_table_contents(dataset[5]["zh"])[-1]))

2

In [13]:
from tqdm import tqdm
from pathlib import Path

index = -1
for row in tqdm(dataset):
    index +=1
    zh_table_contents = get_all_table_contents(row["zh"])
    if zh_table_contents != None:
        
        dir_path = row["record"]
        
        try:
            c = main(row["zh"])
            if c:
                Path(f"./txt/{dir_path}").mkdir(parents=True, exist_ok=True)
                    
                with open(f"./txt/{dir_path}/zh.txt","w") as f:
                    f.write("\n".join(c))
                    
        except Exception:
            print(dir_path)

       
        

  6%|████▊                                                                        | 179/2839 [00:00<00:01, 1729.89it/s]

541602
517535
443393
530555
558888
540906


 17%|████████████▉                                                                | 477/2839 [00:00<00:02, 1038.11it/s]

546983
605507
502499
585857
441135
516439


 27%|████████████████████▉                                                        | 773/2839 [00:00<00:01, 1247.21it/s]

...

3900530
1298910
3970265


 94%|████████████████████████████████████████████████████████████████████████▍    | 2669/2839 [00:02<00:00, 962.44it/s]

3862135
803431
3982858
826551
847080


100%|████████████████████████████████████████████████████████████████████████████| 2839/2839 [00:02<00:00, 1016.29it/s]

3850087
3805736
1483964


In [14]:
from tqdm import tqdm
from pathlib import Path

index = -1
for row in tqdm(dataset):
    index +=1
    zh_table_contents = get_all_table_contents(row["en"])
    if zh_table_contents != None:
        
        dir_path = row["record"]
        
        try:
            c = main(row["en"])
            if c:
                Path(f"./txt/{dir_path}").mkdir(parents=True, exist_ok=True)
                    
                with open(f"./txt/{dir_path}/en.txt","w") as f:
                    f.write("\n".join(c))
                    
        except Exception:
            print(dir_path)

 10%|███████▋                                                                     | 283/2839 [00:00<00:01, 1290.79it/s]

578671
615957
556352
558888
568735
569344
582204


 15%|███████████▏                                                                 | 413/2839 [00:00<00:02, 1081.47it/s]

475369
616031
466395
415094
579650
605507
581261
585857


 30%|██████████████████████▉                                                      | 845/2839 [00:00<00:01, 1315.98it/s]

...

803431
788173
3969036
844753
1473939


 96%|██████████████████████████████████████████████████████████████████████████▏  | 2733/2839 [00:02<00:00, 838.23it/s]

3937519
826551


100%|█████████████████████████████████████████████████████████████████████████████| 2839/2839 [00:03<00:00, 885.00it/s]

1483964
